In [17]:
import glob
import pathlib
import pickle
import numpy as np
import sys
import os
from music21 import converter, instrument, note, chord, stream
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, BatchNormalization, Dropout, Dense, Activation

sys.path.append("..")
from musiclearn import config

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
midi_dir = pathlib.Path(config.FF_MIDI_DIR)

In [3]:
# length of an input sequence
SEQUENCE_LENGTH = 100

In [4]:
def get_notes(midi_dir):
    """ Get all the notes and chords from the midi files in the collection /ff_midi_songs """
    notes = []
    
    noteFile_dir = str(midi_dir) + "/notes"
    
    if not os.path.exists(noteFile_dir):
        files_dir = str(midi_dir) + "/*.mid"
        for file in glob.glob(files_dir):
            midi = converter.parse(file)

            print("Parsing %s" % file)

            notes_to_parse = None

            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse()
            except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                     notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

        # save the features
        with open(noteFile_dir, 'wb') as fp:
            pickle.dump(notes, fp)
    else:
        with open(noteFile_dir, 'rb') as fp:
            notes = pickle.load(fp)
    
    return notes

In [5]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the neural network """
    sequence_length  = 100
    
    # get all pitch name
    pitchnames = sorted(set(item for item in notes))
    
    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    network_input = []
    network_output = []
    
    # create input sequences and teh corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i : i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
        
    n_samples = len(network_input)
    
    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_samples, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    
    network_output = keras.utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [6]:
def lstm_model(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    return model

In [7]:
def train_network(notes, epochs = 20, batch_size = 128):
    """ Train a neural network to generate music """
    # get amount of pitch names
    n_vocab = len(set(notes))
    
    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = lstm_model(network_input, n_vocab)
    
    # train the network
    filepath = "saved_weight.hdf5"
    
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        node='min'
    )
    
    callback_list = [checkpoint_cb]
    
    model.fit(network_input, network_output, epochs=epochs, batch_size=batch_size, callbacks=callback_list)

In [8]:
# extract features
notes = get_notes(midi_dir)

Parsing /csslab-localdata/csslab-si/jack_working_dir/ff_midi_songs/path_of_repentance.mid
Parsing /csslab-localdata/csslab-si/jack_working_dir/ff_midi_songs/FF3_Third_Phase_Final_(Piano).mid
Parsing /csslab-localdata/csslab-si/jack_working_dir/ff_midi_songs/Rydia_pc.mid
Parsing /csslab-localdata/csslab-si/jack_working_dir/ff_midi_songs/balamb.mid
Parsing /csslab-localdata/csslab-si/jack_working_dir/ff_midi_songs/EyesOnMePiano.mid
Parsing /csslab-localdata/csslab-si/jack_working_dir/ff_midi_songs/ff4-airship.mid
Parsing /csslab-localdata/csslab-si/jack_working_dir/ff_midi_songs/ultimafro.mid
Parsing /csslab-localdata/csslab-si/jack_working_dir/ff_midi_songs/DOS.mid
Parsing /csslab-localdata/csslab-si/jack_working_dir/ff_midi_songs/gerudo.mid
Parsing /csslab-localdata/csslab-si/jack_working_dir/ff_midi_songs/FFIII_Edgar_And_Sabin_Piano.mid
Parsing /csslab-localdata/csslab-si/jack_working_dir/ff_midi_songs/roseofmay-piano.mid
Parsing /csslab-localdata/csslab-si/jack_working_dir/ff_midi_so

In [ ]:
# train the model
train_network(notes, 20, 128)

In [9]:
def prepare_sequences_to_predict(notes, n_vocab):
    """ Prepare the sequences to make prediction """
    sequence_length  = 100
    
    # get all pitch name
    pitchnames = sorted(set(item for item in notes))
    
    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    network_input = []
    
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
    
    n_samples = len(network_input)
    
    # reshapre the input
    normalized_input = np.reshape(network_input, (n_samples, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)
    
    return (network_input, normalized_input)

In [10]:
def load_model(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    model.load_weights("saved_weight.hdf5")
    
    return model

In [15]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [12]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output_ff.mid')

In [13]:
def generate_sequence():
    """ Generate a midi file """
    noteFile_dir = str(midi_dir) + "/notes"
    
    # load the notes used to train the model
    with open(noteFile_dir, 'rb') as fp:
        notes = pickle.load(fp)
    
    # get all pitch names
    pitchnames =  sorted(set(item for item in notes))
    # get number of pitches
    n_vocab = len(set(notes))
    
    network_input, normalized_input = prepare_sequences_to_predict(notes, n_vocab)
    
    # load saved weights
    model = load_model(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    # make midi file
    create_midi(prediction_output)

In [18]:
generate_sequence()